In [1]:
import pandas as pd
import urllib3 as urllib
import json

In [2]:
df = pd.read_csv('COVID-19_Nursing_Home_Dataset.tsv', sep='\t')
df_latest = df[df['Week Ending'] == max(df['Week Ending'].unique())]
#df_latest.to_csv('2020-05-31_CMD_DATA.csv')
df['confirmed-occupied_beds'] = df['Residents Total Confirmed COVID-19']/df_latest['Total Number of Occupied Beds']*100
cols = ['Week Ending', 'Federal Provider Number', 'Provider Name',
       'Provider Address', 'Provider City', 'Provider State',
       'Provider Zip Code', 'confirmed-occupied_beds', 
        'Residents Total Suspected COVID-19', 'Residents Total Admissions COVID-19',
        'Residents Total Confirmed COVID-19', 'Total Number of Occupied Beds', 
        'Number of All Beds', 'Total Number of Occupied Beds']
df[(df['confirmed-occupied_beds'] > 100.0) & df['Total Number of Occupied Beds'] > 0][cols].sort_values('confirmed-occupied_beds', ascending=False).to_csv('CMS_Data_More_Cases_Than_Beds.csv')

IL_DF = df[df["Provider State"] == "IL"]

In [3]:
df_latest = df[df['Week Ending'] == max(df['Week Ending'].unique())]
#df_latest.to_csv('2020-05-31_CMD_DATA.csv')

In [4]:
df['confirmed-occupied_beds'] = df['Residents Total Confirmed COVID-19']/df_latest['Total Number of Occupied Beds']*100

In [5]:
cols = ['Week Ending', 'Federal Provider Number', 'Provider Name',
       'Provider Address', 'Provider City', 'Provider State',
       'Provider Zip Code', 'confirmed-occupied_beds', 
        'Residents Total Suspected COVID-19', 'Residents Total Admissions COVID-19',
        'Residents Total Confirmed COVID-19', 'Total Number of Occupied Beds', 
        'Number of All Beds', 'Total Number of Occupied Beds']
df[(df['confirmed-occupied_beds'] > 100.0) & df['Total Number of Occupied Beds'] > 0][cols].sort_values('confirmed-occupied_beds', ascending=False).to_csv('CMS_Data_More_Cases_Than_Beds.csv')

In [6]:
df.head()

,Week Ending,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Submitted Data,Passed Quality Assurance Check,Residents Weekly Admissions COVID-19,...,Ventilator Dependent Unit,Number of Ventilators in Facility,Number of Ventilators in Use for COVID-19,Any Current Supply of Ventilator Supplies,One-Week Supply of Ventilator Supplies,"Total Resident Confirmed COVID-19 Cases Per 1,000 Residents","Total Resident COVID-19 Deaths Per 1,000 Residents",Total Residents COVID-19 Deaths as a Percentage of Confirmed COVID-19 Cases,Geolocation,confirmed-occupied_beds
0,05/24/2020,015141,ARBOR WOODS HEALTH AND REHAB,515 2ND AVENUE NORTHWEST,REFORM,AL,35481.0,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-88.019974 33.381773),NaN
1,05/31/2020,015178,GREENE COUNTY NURSING HOME,509 WILSON AVE,EUTAW,AL,35462.0,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-87.889307 32.844744),NaN
2,05/31/2020,015222,FAIRHOPE HEALTH & REHAB,108 SOUTH CHURCH STREET,FAIRHOPE,AL,36532.0,Y,Y,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,POINT (-87.904817 30.520236),0.0
3,05/31/2020,015416,ANDALUSIA MANOR,670 MOORE RD,ANDALUSIA,AL,36420.0,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-86.497781 31.282418999999997),NaN
4,05/31/2020,035189,LA CANADA CARE CENTER,7970 NORTH LA CANADA DRIVE,TUCSON,AZ,85704.0,N,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-110.99562 32.350849),NaN


In [ ]:
df_latest.columns

In [3]:
#Filter by IL for states


IL_DF[IL_DF['Week Ending'] == '05/31/2020'].sum()[['Residents Total Confirmed COVID-19',  'Residents Total COVID-19 Deaths']]

Residents Total Confirmed COVID-19    7156
Residents Total COVID-19 Deaths       1961
dtype: object

In [4]:
http = urllib.PoolManager()

def getResponse(url):
    operUrl = http.request('GET', url)
    if(operUrl.status==200):
        data = operUrl.data
        jsonData = json.loads(data.decode('utf-8'))
    else:
        print("Error receiving data", operUrl.getcode())
    return jsonData

In [5]:
ltc_data = getResponse('http://www.dph.illinois.gov/sitefiles/COVIDLTC.json')
reporting_date = '%d-%02d-%02d' %(ltc_data['LastUpdateDate']['year'], ltc_data['LastUpdateDate']['month'], ltc_data['LastUpdateDate']['day'])

In [6]:
df1 = pd.DataFrame(ltc_data['FacilityValues'])
df1['reporting_date'] = reporting_date
df1['% deaths'] = df1['deaths'] / df1['confirmed_cases']

In [7]:
df1

,County,FacilityName,confirmed_cases,deaths,status,reporting_date,% deaths
0,Boone,Park Place of Belvidere,5,0,Open,2020-06-05,0.000000
1,Boone,Symphony Northwoods,87,13,Open,2020-06-05,0.149425
2,Champaign,Reflections Memory Care,8,0,Open,2020-06-05,0.000000
3,Champaign,University Rehab Center,5,0,Open,2020-06-05,0.000000
4,Christian,Villas of Holly Brook,2,0,Open,2020-06-05,0.000000
...,...,...,...,...,...,...,...
549,Winnebago,Peterson Meadows,7,1,Open,2020-06-05,0.142857
550,Winnebago,River Bluff Nursing Home,5,0,Open,2020-06-05,0.000000
551,Winnebago,Rock River Health Care,9,0,Open,2020-06-05,0.000000
552,Winnebago,Stepping Stone of Rockford,7,1,Open,2020-06-05,0.142857


In [8]:
f = open("../Covid-County-LTC-Analysis/Source_data/IL_2020-05-29LTC_data_Source.json")

FileNotFoundError: [Errno 2] No such file or directory: '../Covid-County-LTC-Analysis/Source_data/IL_2020-05-29LTC_data_Source.json'